# 03 CIC, Forces, and Diagnostics

This notebook demonstrates the particle-mesh mapping steps: CIC density deposition, Poisson-based force computation, and diagnostic summaries.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / "src").exists() and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("Using src path:", SRC)

In [ ]:
from lcdm_sim.config import load_simulation_config
from lcdm_sim.grf import generate_grf
from lcdm_sim.zeldovich import initial_conditions_from_density
from lcdm_sim.cic import deposit_density, interpolate_forces_to_particles
from lcdm_sim.forces import compute_comoving_acceleration_grid
from lcdm_sim.diagnostics import density_stats, acceleration_stats, estimate_power_spectrum, compare_cic_to_initial
import numpy as np

cfg = load_simulation_config(ROOT / "configs" / "smoke.yaml")

In [ ]:
delta0 = generate_grf(cfg)
state0 = initial_conditions_from_density(delta0, cfg)
delta_cic = deposit_density(state0, cfg.grid)
acc_grid = compute_comoving_acceleration_grid(delta_cic, cfg)
acc_particles = interpolate_forces_to_particles(acc_grid, state0, cfg.grid)
print("CIC density stats:", density_stats(delta_cic))
print("Accel stats:", acceleration_stats(acc_grid))
print("CIC vs initial:", compare_cic_to_initial(delta0, delta_cic))
print("Particle accel shape:", acc_particles.shape)

In [ ]:
ps = estimate_power_spectrum(delta_cic, nbins=12)
print("Power bins:", len(ps["k_centers"]))
ps

In [ ]:
import matplotlib.pyplot as plt
from lcdm_sim.plotting_static import plot_density_projection

# Display quick inline projection from current field
proj = delta_cic.data.sum(axis=2)
plt.figure(figsize=(5,4))
plt.imshow(proj.T, origin="lower", cmap="viridis")
plt.title("CIC density projection")
plt.colorbar()
plt.show()